**Tensor**是pytorch中的重要变量，包含具体的数值和梯度两部分，当loss执行backward时梯度部分会计算loss对于w权重的偏导
**LongTensor**常常用于离散的数据和标签；部分时候输入要求必须是**LongTensor**，如果不对会报错，改正即可

In [ ]:
import torch

w = torch.Tensor([1.0])
w.requires_grad = True

损失**loss**执行反向传播通过backward函数实现，注意该函数在没有添加智能补全插件的其前提下不会出现补全提示

In [ ]:
loss.backward()

获取**Tensor**中的数值，可以直接访问（得到tensor类型数据），或者加上**item()**转化为int或float类型的数值，当前**grad**需要执行反向传播才会有数据否则报错

In [ ]:
import torch

x = torch.randn(3, 3, requires_grad=True)
print("Tensor data:", x)

# 使用 x 直接访问数据
print(x)

# 或者通过 .data 访问
print(x.data)

print(x.grad.item())

Tensor data: tensor([[-0.8346, -0.1115,  0.0908],
        [-0.3981,  1.4364, -0.9879],
        [ 2.0842, -0.6768,  1.3137]], requires_grad=True)
tensor([[-0.8346, -0.1115,  0.0908],
        [-0.3981,  1.4364, -0.9879],
        [ 2.0842, -0.6768,  1.3137]], requires_grad=True)
tensor([[-0.8346, -0.1115,  0.0908],
        [-0.3981,  1.4364, -0.9879],
        [ 2.0842, -0.6768,  1.3137]])


梯度**grad**在每次执行完权重更新后需要归零，否则pytorch默认的计算方式是梯度累加的（当然部分模型结构确实是用累加梯度的）

In [ ]:
w.grad.data.zero_() #我自定义一个tensor作为权重w（可视为一个极其简单的无偏置网络层）并将梯度归零
optimizer.zero_grad() #实际使用过程中会定义优化器optimizer及其用zero_grad()方法归零权重

**torch.nn.Linear(m, n)**包含权重和偏置，权重为[n,m]的矩阵，偏置为一维的[n,1]/[n,]

**forward**是魔法函数，简单来说，后期如果需要调用实例化的模型进行前馈计算，不需要**model.forward(x)**这种形式而是直接**model(x)**即可

**sigmod/ReLU**这类激活函数，可以来自于**torch.nn.functional**也可以直接**torch.nn.Sigmoid()**调用

In [ ]:
class LinearModel(torch.nn.Module):
    def __init__(self):
        super(LinearModel, self).__init__() #just do it
        self.linear = torch.nn.Linear(1, 1)
    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred

model = LinearModel()

# 查看具体参数
print('w = ', model.linear.weight.item())
print('b = ', model.linear.bias.item())

定义损失函数与优化器，替代了我们自行定义设计的损失函数和梯度下降函数（一般为最小二乘法那个的导数）

In [ ]:
criterion = torch.nn.MSELoss(reduction='sum')  # 或 'mean'，视需要而定
criterion = torch.nn.BCELoss(reduction='sum') # 交叉熵损失

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

训练基本流程

In [ ]:
for epoch in range(100):

    y_pred = model(x_data) #准备输入数据和标签

    loss = criterion(y_pred, y_data) #使用criterion计算损失

    optimizer.zero_grad() #权重梯度归零；部分情况可能不需要

    loss.backward() #计算损失，本质上损失的数值不重要，重要的是通过这一步完成计算图的反向传播构建，并赋予optimizer新的权重梯度

    optimizer.step() #这一步是完成权重更新的计算

读取/下载数据
1.pytorch中自带的数据集下载
**transform= transforms.ToTensor()**用于将数据转为**Tensor**
2.自己的数据导入
要想将自定义的 Dataset 作为 DataLoader 的输入参数，必须确保它实现了以下两个方法：
__getitem__(self, index)：这个方法应该返回数据集的每个样本。返回值通常是一个元组 (x, y)，其中 x 是输入数据，y 是标签或目标数据。通过这个方法，DataLoader 可以按需获取每一批次的数据。
__len__(self)：返回数据集的总大小。DataLoader 需要知道数据集的大小来进行分批次加载。
只要这两个方法符合要求，Dataset 就可以作为 DataLoader 的输入。


In [ ]:
import torchvision
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms

train_set = torchvision.datasets.MNIST(root='./dataset/mnist', train=True, download=True)
test_set = torchvision.datasets.MNIST(root='./dataset/mnist', train=False, download=True)
# train_set = torchvision.datasets.CIFAR10(…)
# test_set = torchvision.datasets.CIFAR10(…)
test_set = torchvision.datasets.MNIST(root='./dataset/mnist', train=False,transform= transforms.ToTensor(), download=True)


class DiabetesDataset(Dataset):
    def __init__(self):
        pass
    def __getitem__(self, index):
        pass
    def __len__(self):
        pass
dataset = DiabetesDataset()
train_loader = DataLoader(dataset=dataset,
                            batch_size=32,
                            shuffle=True,
                            num_workers=2) #启用多线程读取，该维度一般建议删除会报错

# 往往配合如下
for epoch in range(100):
    for i, data in enumerate(train_loader, 0):  #如果默认从第一个batch读取，0可以删除
        pass

断点测试模型各层维度

关于**Softmax**的逻辑补充，如下图所示

![image.png](Softmax.png)

分类计算的结果经过转换后与目标标签的one-hot编码进行交叉熵计算，本质上只有one-hot值为1的维度及其对应的概率值进行了有效计算，也就是损失就是只有如下一个数值
$$ -\log(\hat{Y}) $$

仔细比较下方手撸的交叉熵和官方提供的交叉熵会发现，实际计算过程中目标标签只需要给出一个值而不是一个one-hot的数组即可。因为交叉熵函数会根据给出的值（就是具体代表的类别）作为下标去取预测的数组中对应维度的那个值，然后就足以计算损失了

In [ ]:
import numpy as np

y = np.array([1, 0, 0])
z = np.array([0.2, 0.1, -0.1])
y_pred = np.exp(z) / np.exp(z).sum()
loss = (- y * np.log(y_pred)).sum()
print(loss)

y = torch.LongTensor([0])
z = torch.Tensor([[0.2, 0.1, -0.1]])
criterion = torch.nn.CrossEntropyLoss()
loss = criterion(z, y)
print(loss)


训练前对数据归一化的好处
在深度学习中，**正态分布**或接近 **标准正态分布**（均值为 0，标准差为 1）的数据对加速训练过程非常有利。这与模型的优化、梯度下降算法的表现以及神经网络的训练过程密切相关。以下是一些关键原因，解释为什么数据符合正态分布能够加速训练：

### 1. **避免梯度爆炸或梯度消失**
神经网络在训练过程中依赖反向传播算法计算梯度，而梯度的大小直接影响权重更新的步长。

- **梯度爆炸**：如果输入数据的范围太大，网络中的梯度在反向传播过程中可能会变得非常大，导致权重更新过大，进而导致数值不稳定，甚至无法收敛。
- **梯度消失**：如果输入数据的范围过小，梯度可能会变得非常小，导致权重更新非常缓慢，甚至没有有效的更新。

通过将数据转换为零均值、单位方差的分布（即标准正态分布），可以有效地避免这两种问题。标准化后的数据避免了输入数据的极端值，从而减少了梯度爆炸或梯度消失的风险，使得训练过程更加稳定。

### 2. **加快收敛速度**
当输入数据符合正态分布时，神经网络模型的训练过程往往能更快地收敛。这是因为：

- **标准化后的数据具有统一的尺度**：标准化后，所有输入特征的范围变得一致（均值为 0，标准差为 1）。这意味着在梯度下降过程中，每个特征对梯度的贡献相对均衡，避免了某些特征对模型训练产生过大的影响。
- **梯度下降更加高效**：统一的尺度让优化器（如SGD、Adam等）能够更均匀地更新网络权重，从而更高效地找到最优解。否则，如果特征尺度不一致，优化器可能会沿某些维度“过冲”或“滞后”，导致收敛速度减慢。

### 3. **提高模型的鲁棒性**
神经网络模型的鲁棒性是指其在面对不同训练数据或输入时的表现稳定性。标准化后的数据：
- **避免模型对某些特征过度敏感**：如果某些特征的数值范围过大，网络可能会对这些特征过度敏感，而忽视其他重要特征。通过标准化，所有特征的影响力被平衡，使模型能够全面地学习每个特征。
- **提高泛化能力**：数据的标准化有助于减少过拟合，并使模型更容易在不同数据分布下表现得更好。标准化后的数据通常使模型在新数据上的表现更加稳定，具有更好的泛化能力。

### 4. **平衡权重更新**
神经网络中的每一层通常由许多神经元组成，每个神经元的输出通常是前一层输出的加权和（加上偏置）。如果输入数据的分布不均匀，某些神经元的加权和可能会非常大或非常小，从而导致训练时权重更新的不平衡。标准化数据确保所有神经元在训练开始时得到一个均衡的起点，避免了部分神经元在训练过程中过度主导模型更新。

### 5. **对优化算法的适应性**
许多优化算法（如 Adam, RMSprop, etc.）在数据标准化时表现得更好。这些优化器通常会根据输入数据的特性（如均值、方差）调整学习率和梯度的更新步伐。通过对数据进行标准化处理，优化算法能够更高效地适应不同的任务和数据特性，从而提高训练速度。

### 6. **减少学习率调节的需求**
未标准化的数据可能需要更复杂的学习率调节机制，以便避免训练过程中出现过大的梯度更新。标准化后的数据通常允许你使用一个固定的学习率，而不需要频繁调整，从而简化了超参数的调节过程。

---

### 总结
符合正态分布的数据对于加速训练非常有利，因为它能：
- **避免梯度爆炸和梯度消失**，确保梯度计算更加稳定；
- **加速收敛**，通过标准化后的数据，优化器能够更高效地进行权重更新；
- **提高模型的鲁棒性**，减少对单个特征的过度依赖，提升模型在不同数据集上的泛化能力；
- **平衡权重更新**，避免某些神经元过度主导训练过程；
- **简化优化算法的调节**，减少学习率调节的复杂性。

因此，标准化处理（尤其是标准正态分布的处理）是深度学习中数据预处理的一个重要步骤，能够显著提升模型训练的效率和效果。

In [ ]:
transform = transforms.Compose([
transforms.ToTensor(),
transforms.Normalize((0.1307, ), (0.3081, ))
])

# 以MNIST为例

NameError: name 'transforms' is not defined

在测试阶段使用**with torch.no_grad():**避免梯度计算减少不必要的开销